## About the Data

> The dataset is available as a list with 146,028 dyads of players and referees and includes details from players, details from referees and details regarding the interactions of player-referees. A summary of the variables of interest can be seen below. A detailed description of all variables included can be seen in the README file on the project website. 

> From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2,053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3,147) that these players played under in their professional career (see Figure 1). We created a dataset of playerâreferee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.

> -- https://docs.google.com/document/d/1uCF5wmbcL90qvrk_J27fWAvDcDNrO9o_APkicwRkOKc/edit


| Variable Name: | Variable Description: | 
| -- | -- | 
| playerShort | short player ID | 
| player | player name | 
| club | player club | 
| leagueCountry | country of player club (England, Germany, France, and Spain) | 
| height | player height (in cm) | 
| weight | player weight (in kg) | 
| position | player position | 
| games | number of games in the player-referee dyad | 
| goals | number of goals in the player-referee dyad | 
| yellowCards | number of yellow cards player received from the referee | 
| yellowReds | number of yellow-red cards player received from the referee | 
| redCards | number of red cards player received from the referee | 
| photoID | ID of player photo (if available) | 
| rater1 | skin rating of photo by rater 1 | 
| rater2 | skin rating of photo by rater 2 | 
| refNum | unique referee ID number (referee name removed for anonymizing purposes) | 
| refCountry | unique referee country ID number | 
| Alpha_3 | country name in short form|
| meanIAT | mean implicit bias score (using the race IAT) for referee country | 
| nIAT | sample size for race IAT in that particular country | 
| seIAT | standard error for mean estimate of race IAT   | 
| meanExp | mean explicit bias score (using a racial thermometer task) for referee country | 
| nExp | sample size for explicit bias in that particular country | 
| seExp |  standard error for mean estimate of explicit bias measure | 

## The Task

Do an Exploratory Data Analysis on the redcard dataset. Keeping in mind the question is the following: **Are soccer referees more likely to give red cards to dark-skin-toned players than light-skin-toned players?**

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
sns.set_context("poster", font_scale=1.3)

import missingno as msno
import pandas_profiling

from sklearn.datasets import make_blobs
import time

In [2]:
df = pd.read_csv('redcard.csv.gz', compression = 'gzip')

df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [3]:
df.describe()

,height,weight,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,rater1,rater2,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
count,145765.000000,143785.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,124621.000000,124621.000000,146028.000000,146028.000000,145865.000000,1.458650e+05,1.458650e+05,145865.000000,1.458650e+05,145865.000000
mean,181.935938,76.075662,2.921166,1.278344,0.708241,0.934581,0.338058,0.385364,0.011381,0.012559,0.264255,0.302862,1534.827444,29.642842,0.346276,1.969741e+04,6.310849e-04,0.452026,2.044023e+04,0.002994
std,6.738726,7.140906,3.413633,1.790725,1.116793,1.383059,0.906481,0.795333,0.107931,0.112889,0.295382,0.293020,918.736625,27.496189,0.032246,1.271262e+05,4.735857e-03,0.217469,1.306157e+05,0.019723
min,161.000000,54.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-0.047254,2.000000e+00,2.235373e-07,-1.375000,2.000000e+00,0.000001
25%,177.000000,71.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,641.000000,7.000000,0.334684,1.785000e+03,5.454025e-05,0.336101,1.897000e+03,0.000225
50%,182.000000,76.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,1604.000000,21.000000,0.336628,2.882000e+03,1.508847e-04,0.356446,3.011000e+03,0.000586
75%,187.000000,81.000000,3.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.250000,0.500000,2345.000000,44.000000,0.369894,7.749000e+03,2.294896e-04,0.588297,7.974000e+03,0.001002
max,203.000000,100.000000,47.000000,29.000000,14.000000,18.000000,23.000000,14.000000,3.000000,2.000000,1.000000,1.000000,3147.000000,161.000000,0.573793,1.975803e+06,2.862871e-01,1.800000,2.029548e+06,1.060660


### Now the DataFrame is arranged such that every observation is recorded as a player and referee interaction. Therefore, in the player column, the same player/playerShort may appear more than once when they interact with different referees

> Therefore, we find the data is not TIDY at all, there are multiple observational units in one table. Our job would be to separate these and such that each variable forms a column, each observation forms a row and each type of observational unit forms it's own table. This is the concept of TIDY data by Hadly WickHam

In [4]:
df.columns

Index(['playerShort', 'player', 'club', 'leagueCountry', 'birthday', 'height',
       'weight', 'position', 'games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards', 'photoID', 'rater1', 'rater2',
       'refNum', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp'],
      dtype='object')

## Tidy Players Table

In [5]:
player_index = ['playerShort']
playerCols = [
              'birthday',
              'position',
              'height',
              'weight',
              'photoID',
              'rater1',
              'rater2'] 

In [6]:
def create_subFrame(dataFrame, tableIndex, tableColumns):
    
    check_unique_table = dataFrame.groupby(tableIndex).agg({col:'nunique' for col in dataFrame[tableColumns].columns})
    if (check_unique_table[check_unique_table>1].dropna().shape[0] == 0):
        print ('There are no discrepancies in data')
    else:
        print (' Warning! there is some uniqueness in data')
        
    return dataFrame.groupby(tableIndex).agg({col: 'min' for col in dataFrame[tableColumns].columns})

In [7]:
def save_subFrame(dataFrame, g_index, name = 'default'):
    subFrame_name = ''.join([name, '.csv.gz'])
    dataFrame.to_csv(subFrame_name, compression='gzip', encoding = 'UTF8')
 

In [8]:
players = create_subFrame(df, player_index, playerCols)

There are no discrepancies in data


In [9]:
players.head()

,birthday,position,height,weight,photoID,rater1,rater2
playerShort,,,,,,,
aaron-hughes,08.11.1979,Center Back,182.0,71.0,3868.jpg,0.25,0.00
aaron-hunt,04.09.1986,Attacking Midfielder,183.0,73.0,20136.jpg,0.00,0.25
aaron-lennon,16.04.1987,Right Midfielder,165.0,63.0,13515.jpg,0.25,0.25
aaron-ramsey,26.12.1990,Center Midfielder,178.0,76.0,94953.jpg,0.00,0.00
abdelhamid-el-kaoutari,17.03.1990,Center Back,180.0,73.0,124913.jpg,0.25,0.25


In [10]:
players.describe()

,height,weight,rater1,rater2
count,2033.000000,1973.000000,1585.000000,1585.00000
mean,181.743728,75.639635,0.268612,0.31041
std,6.690114,7.101795,0.296578,0.29666
min,161.000000,54.000000,0.000000,0.00000
25%,177.000000,71.000000,0.000000,0.00000
50%,182.000000,75.000000,0.250000,0.25000
75%,186.000000,80.000000,0.500000,0.50000
max,203.000000,100.000000,1.000000,1.00000


In [11]:
save_subFrame(players, player_index, name = 'players')

## Tidy Clubs table

In [12]:
club_index = ['club']
club_columns = ['leagueCountry']

In [13]:
clubs = create_subFrame(df, club_index, club_columns)

There are no discrepancies in data


In [14]:
clubs.reset_index(inplace = True)

In [15]:
clubs['leagueCountry'].value_counts()

England    48
Spain      27
France     22
Germany    21
Name: leagueCountry, dtype: int64

In [16]:
clubs.head()

,club,leagueCountry
0,1. FC Nürnberg,Germany
1,1. FSV Mainz 05,Germany
2,1899 Hoffenheim,Germany
3,AC Ajaccio,France
4,AFC Bournemouth,England


In [17]:
save_subFrame(clubs, club_index, name = 'clubs')

## Tidy Referee table

In [18]:
ref_index = ['refNum']
ref_col = ['refCountry']

In [19]:
referees = create_subFrame(df, ref_index, ref_col)

There are no discrepancies in data


In [20]:
referees.head()

,refCountry
refNum,
1,1
2,2
3,3
4,4
5,5


In [21]:
referees.tail()

,refCountry
refNum,
3143,51
3144,55
3145,21
3146,51
3147,21


In [22]:
save_subFrame(referees, ref_index, name = 'referees')

## Tidy Countries table

In [23]:
countries_index = ['refCountry']
country_columns = [ 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT',
       'meanExp', 'nExp', 'seExp']

In [24]:
countries = create_subFrame(df, countries_index, country_columns)

There are no discrepancies in data


In [25]:
countries.head()

,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
refCountry,,,,,,,
1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
5,TUN,0.167132,19.0,0.027327,-0.789474,19.0,0.111757


In [26]:
countries.rename(columns = {'Alpha_3': 'country_name'}, inplace = True)

In [27]:
countries.head()

,country_name,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
refCountry,,,,,,,
1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
5,TUN,0.167132,19.0,0.027327,-0.789474,19.0,0.111757


In [28]:
save_subFrame(countries, countries_index, name = 'countries')

## Creating Dyadas table

In [29]:
dyad_index = ['refNum', 'playerShort']
dyad_cols = ['games', 'victories', 'ties', 'defeats', 'goals',
       'yellowCards', 'yellowReds', 'redCards']

In [30]:
dyads = create_subFrame(df, dyad_index, dyad_cols)

There are no discrepancies in data


In [31]:
dyads.head()

games  victories  ties  defeats  goals  yellowCards  \
refNum playerShort                                                          
1      lucas-wilchez      1          0     0        1      0            0   
2      john-utaka         1          0     0        1      0            1   
3      abdon-prats        1          0     1        0      0            1   
       pablo-mari         1          1     0        0      0            0   
       ruben-pena         1          1     0        0      0            0   

                      yellowReds  redCards  
refNum playerShort                          
1      lucas-wilchez           0         0  
2      john-utaka              0         0  
3      abdon-prats             0         0  
       pablo-mari              0         0  
       ruben-pena              0         0

In [32]:
save_subFrame(dyads, dyad_index, name = 'dyads')